In [1]:
import torch
torch.cuda.empty_cache()
print(torch.__version__)#查看torch版本号
print(torch.cuda.is_available())  # 应该返回 True
print(torch.cuda.current_device())  # 返回当前使用的GPU编号
print(torch.cuda.get_device_name(0))  # 返回当前设备的名称

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


2.2.1+cu121
True
0
NVIDIA GeForce RTX 3060 Laptop GPU


In [2]:
from transformers import AutoModel, AutoTokenizer

# 下载并保存模型和tokenizer
path_to_save_model = r'C:/大语言模型库/original_model/BAAI/bge-base-zh-v1.5'
model = AutoModel.from_pretrained("BAAI/bge-base-zh-v1.5")
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-base-zh-v1.5")

model.save_pretrained(path_to_save_model)
tokenizer.save_pretrained(path_to_save_model)

pytorch_model.bin:  69%|######9   | 283M/409M [00:00<?, ?B/s]

E:\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\86131\.cache\huggingface\hub\models--BAAI--bge-base-zh-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

('C:/大语言模型库/original_model/BAAI/bge-base-zh-v1.5\\tokenizer_config.json',
 'C:/大语言模型库/original_model/BAAI/bge-base-zh-v1.5\\special_tokens_map.json',
 'C:/大语言模型库/original_model/BAAI/bge-base-zh-v1.5\\vocab.txt',
 'C:/大语言模型库/original_model/BAAI/bge-base-zh-v1.5\\added_tokens.json',
 'C:/大语言模型库/original_model/BAAI/bge-base-zh-v1.5\\tokenizer.json')

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 第一步：导入Excel文件为DataFrame
file_path = r"C:\Users\86131\Desktop\科大讯飞实习\分享 8000个问答对全文件-罗鑫\问答对8000条包含source和ai_message汇总.xlsx"
df = pd.read_excel(file_path)


# 第二步：提取第三列和第五列，生成df1
df1 = df.iloc[:, [2, 4]]  # 第三列和第五列

# 第三步：删除第一列值为 "Question" 的所有行，生成df2
df2 = df1[df1.iloc[:, 0] != "Question"]

# 第四步：划分为训练集和测试集（80% 训练集，20% 测试集）
train_df, test_df = train_test_split(df2, test_size=0.2, random_state=42)

# 第五步：将训练集和测试集转换为指定格式
train_queries_list = [ row[0] for row in train_df.to_numpy() ]
train_docs_list = [ row[1] for row in train_df.to_numpy() ]
train_relevant_docs_indices = [[i] for i in range(len(train_df.to_numpy()))]

test_queries_list = [ row[0] for row in test_df.to_numpy() ]
test_docs_list = [ row[1] for row in test_df.to_numpy() ]
test_relevant_docs_indices = [[i] for i in range(len(test_df.to_numpy()))]

'''
# 输出 train_set 和 test_set 的部分数据以验证
print("Train Set Example:", train_dataset[:2])
print("Test Set Example:", test_dataset[:2])'''
 

E:\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


'\n# 输出 train_set 和 test_set 的部分数据以验证\nprint("Train Set Example:", train_dataset[:2])\nprint("Test Set Example:", test_dataset[:2])'

In [3]:
def convert_to_dataset_format(queries_list, docs_list, relevant_docs_indices):
    # 创建查询字典
    queries = {str(i+1): query for i, query in enumerate(queries_list)}
    
    # 创建文档字典
    corpus = {f"doc{str(i+1)}": doc for i, doc in enumerate(docs_list)}
    
    # 创建相关文档字典
    relevant_docs = {
        str(i+1): [f"doc{str(idx+1)}" for idx in indices] 
        for i, indices in enumerate(relevant_docs_indices)
    }
    
    return {
        "queries": queries,
        "corpus": corpus,
        "relevant_docs": relevant_docs
    }

# 转换为适合的格式
train_dataset_dict = convert_to_dataset_format(train_queries_list, train_docs_list, train_relevant_docs_indices)
val_dataset_dict = convert_to_dataset_format(test_queries_list,test_docs_list,test_relevant_docs_indices)
 

In [4]:
class FinetuneDataset:
    def __init__(self, queries, relevant_docs, corpus):
        self.queries = queries
        self.relevant_docs = relevant_docs
        self.corpus = corpus

# Wrap it in the FinetuneDataset class
train_dataset = FinetuneDataset(
    queries=train_dataset_dict["queries"],
    relevant_docs=train_dataset_dict["relevant_docs"],
    corpus=train_dataset_dict["corpus"]
)
val_dataset = FinetuneDataset(
    queries=val_dataset_dict["queries"],
    relevant_docs=val_dataset_dict["relevant_docs"],
    corpus=val_dataset_dict["corpus"]
)

In [12]:
path_to_save_finetuned_model = r'C:/大语言模型库/Finetuned_model/BAAI/bge-base-zh-v1.5(4)'
#path_to_save_finetuned_model = r'C:/大语言模型库/Finetuned_model/BAAI/bge-small-zh-v1.5'
from llama_index.finetuning import SentenceTransformersFinetuneEngine
path_to_load_model = r'C:/大语言模型库/Finetuned_model/BAAI/bge-base-zh-v1.5(3)'
#path_to_load_model = r"C:/大语言模型库/original_model/BAAI/bge-small-zh-v1.5"
finetune_engine = SentenceTransformersFinetuneEngine(
    train_dataset,#训练集
    model_id= path_to_load_model,  # 本地路径
    model_output_path = path_to_save_finetuned_model, # 微调后模型保存路径
    val_dataset=val_dataset#测试集
)

finetune_engine.finetune()
finetuned_embed_model = finetune_engine.get_finetuned_model()


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: C:/大语言模型库/Finetuned_model/BAAI/bge-base-zh-v1.5(3)
Load pretrained SentenceTransformer: C:/大语言模型库/Finetuned_model/BAAI/bge-base-zh-v1.5(3)
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
Use pytorch device_name: cuda


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/692 [00:00<?, ?it/s]

INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Information Retrieval Evaluation of the model on the  dataset in epoch 0 after 50 steps:
Information Retrieval Evaluation of the model on the  dataset in epoch 0 after 50 steps:
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Queries: 1731
Queries: 1731
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Corpus: 1731

Corpus: 1731

INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Score-Function: cos_sim
Score-Function: cos_sim
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@1: 74.87%
Accuracy@1: 74.87%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@3: 89.77%
Accuracy@3: 89.77%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@5: 92.84%
Accuracy@5: 92.84%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@10: 96.36%
Accuracy@10: 96.36%
INFO:sentence_transf

Iteration:   0%|          | 0/692 [00:00<?, ?it/s]

INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Information Retrieval Evaluation of the model on the  dataset in epoch 1 after 50 steps:
Information Retrieval Evaluation of the model on the  dataset in epoch 1 after 50 steps:
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Queries: 1731
Queries: 1731
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Corpus: 1731

Corpus: 1731

INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Score-Function: cos_sim
Score-Function: cos_sim
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@1: 73.77%
Accuracy@1: 73.77%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@3: 88.50%
Accuracy@3: 88.50%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@5: 92.14%
Accuracy@5: 92.14%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@10: 96.07%
Accuracy@10: 96.07%
INFO:sentence_transf

In [7]:
finetuned_embed_model

HuggingFaceEmbedding(model_name='C:/大语言模型库/Finetuned_model/BAAI/bge-base-zh-v1.5(3)', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000001171ACF2B10>, num_workers=None, max_length=512, normalize=True, query_instruction=None, text_instruction=None, cache_folder=None)

In [8]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core.schema import TextNode
from tqdm.notebook import tqdm
import pandas as pd

In [9]:
def evaluate(
    dataset,#数据集
    embed_model,#模型文件路径
    top_k=5,#检索chunk数
    verbose=False,
):
    corpus = dataset.corpus#加载语料库
    queries = dataset.queries#加载查询
    relevant_docs = dataset.relevant_docs#加载查询与语料库的相关关系
    
    #语料库使用嵌入模型转化为向量存入向量数据库构建向量检索器。
    nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items()]
    index = VectorStoreIndex(
        nodes, embed_model=embed_model, show_progress=True
    )
    retriever = index.as_retriever(similarity_top_k=top_k)
    
    eval_results = []
    for query_id, query in tqdm(queries.items()):
        #检索查询，得到相关的chunk，
        retrieved_nodes = retriever.retrieve(query)
        #相关chunks取Top1,比对查询得到的chunk与标准答案里面的chunk是否相同
        retrieved_ids = [node.node.node_id for node in retrieved_nodes]
        expected_id = relevant_docs[query_id][0]
        is_hit = expected_id in retrieved_ids  # assume 1 relevant doc

        eval_result = {
            "is_hit": is_hit,
            "retrieved": retrieved_ids,
            "expected": expected_id,
            "query": query_id,
        }
        eval_results.append(eval_result)
    return eval_results

In [10]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers import SentenceTransformer
from pathlib import Path


def evaluate_st(
    dataset,#数据集
    model_id,#模型文件地址
    name,#
):
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs

    evaluator = InformationRetrievalEvaluator(
        queries, corpus, relevant_docs, name=name
    )
    model = SentenceTransformer(model_id)
    output_path = "results/"
    Path(output_path).mkdir(exist_ok=True, parents=True)
    return evaluator(model, output_path=output_path)

原始bge-small-zh-v1.5的测评结果

In [10]:
bge = r"local:C:\大语言模型库\original_model\BAAI\bge-small-zh-v1.5"
bge_val_results = evaluate(val_dataset, bge)

No sentence-transformers model found with name C:\大语言模型库\original_model\BAAI\bge-small-zh-v1.5. Creating a new one with MEAN pooling.


Generating embeddings:   0%|          | 0/1731 [00:00<?, ?it/s]

E:\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


  0%|          | 0/1731 [00:00<?, ?it/s]

In [11]:
df_bge = pd.DataFrame(bge_val_results)

In [14]:
hit_rate_bge = df_bge["is_hit"].mean()
hit_rate_bge

0.7920277296360485

bge-zh-small-v1.5微调后测评结果

In [15]:
finetuned = "local:C:/大语言模型库/Finetuned_model/BAAI/bge-small-zh-v1.5"
val_results_finetuned = evaluate(val_dataset, finetuned)

Generating embeddings:   0%|          | 0/1731 [00:00<?, ?it/s]

  0%|          | 0/1731 [00:00<?, ?it/s]

In [18]:
df_finetuned = pd.DataFrame(val_results_finetuned)

In [19]:
hit_rate_finetuned = df_finetuned["is_hit"].mean()
hit_rate_finetuned

0.8908145580589255

原始的Bge-base-zh-v1.5的效果

In [9]:
bge = r"local:C:\大语言模型库\original_model\BAAI\bge-base-zh-v1.5"
bge_val_results = evaluate(val_dataset, bge)
df_result = pd.DataFrame(bge_val_results)
hit_rate = df_result["is_hit"].mean()
hit_rate

No sentence-transformers model found with name C:\大语言模型库\original_model\BAAI\bge-base-zh-v1.5. Creating a new one with MEAN pooling.


Generating embeddings:   0%|          | 0/1731 [00:00<?, ?it/s]

E:\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


  0%|          | 0/1731 [00:00<?, ?it/s]

0.7452339688041595

bge-base-zh-v1.5微调后的测评结果

In [13]:
bge = r"local:C:\大语言模型库\Finetuned_model\BAAI\bge-base-zh-v1.5(4)"
bge_val_results = evaluate(val_dataset, bge)
df_result = pd.DataFrame(bge_val_results)
hit_rate = df_result["is_hit"].mean()
hit_rate

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: C:\大语言模型库\Finetuned_model\BAAI\bge-base-zh-v1.5(4)
Load pretrained SentenceTransformer: C:\大语言模型库\Finetuned_model\BAAI\bge-base-zh-v1.5(4)
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


Generating embeddings:   0%|          | 0/1731 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.9243212016175621